In [ ]:
%matplotlib inline
import os, shutil
import sys
sys.path.append("..")
import numpy as np
from IPython.display import Image
import pandas as pd
import matplotlib.pyplot as plt

import flopy as flopy
import pyemu

# Pst from input/output files

In [ ]:
import freyberg_setup as fs
pst_name = fs.PST_NAME_KR
working_dir = fs.WORKING_DIR_KR
fs.setup_pest_kr()
pst = pyemu.Pst(os.path.join(working_dir,pst_name))
pst.control_data.noptmax = 0
pst.observation_data.loc[pst.observation_data.obgnme=='calflux', 'weight']=0

pst.write(os.path.join(working_dir,pst_name))


In [ ]:
# get 4 lists from the working directory: tpl_files, input_files, ins_files, out_files
io_files = pyemu.helpers.parse_dir_for_io_files(working_dir)
io_files

In [ ]:

os.chdir(working_dir) # change dir into the working dir
pst = pyemu.helpers.pst_from_io_files(*io_files) #get a pst instance using the io files
os.chdir("..") # change dir back out of the working dir
pst.parameter_data 
pst.write("temp.pst")
pst.add_parameters()

# from zero to hero with MODFLOW

load the existing model

In [ ]:
nam = 'freyberg.truth.nam'
model_ws = os.path.join("..","..","models","Freyberg","Freyberg_Truth")

In [ ]:
m = flopy.modflow.Modflow.load(nam,model_ws=model_ws,check=False,exe_name="mfnwt")
m.name = "freyberg"

change the workspace, write inputs and run the model

In [ ]:
m.change_model_ws("temp")
m.write_input()
m.run_model()

# So now what???

- props
- spatial bc props
- temporal bc props
- plot prior cov
- plot residuals
- monte carlo
- 


In [ ]:
# what properties in the model we want to parameterize - 
#3D things, the number is the layer index
#for 2D transient things, the number is the stress period index
props = [["upw.hk",0],["rch.rech",0],["rch.rech",[1,2]]]

In [ ]:
# add multipliers for wel flux in the layer 1
bc_props = [["wel.flux",0]]
# add observations from every active cell in layer 1, stress period 1
hds_kperk=[[0,0]]

In [ ]:
ph = pyemu.helpers.PstFromFlopyModel("freyberg.nam",org_model_ws="temp",
                                    new_model_ws="template",pp_props=props,
                                    const_props=props,spatial_bc_props=bc_props,
                                    remove_existing=True,pp_space=3,
                                          hds_kperk=hds_kperk)

In [ ]:
ph.pst.parameter_data.shape

In [ ]:
ph.pst.observation_data.shape

In [ ]:
x = ph.parcov.x

In [ ]:
plt.imshow(np.ma.masked_where(x==0,x))

In [ ]:
pyemu.helpers.zero_order_tikhonov(ph.pst)

In [ ]:
ph.pst.prior_information.loc["hk003","weight"] = 10.0

In [ ]:
ph.pst.prior_information

In [ ]:
pe = pyemu.ParameterEnsemble.from_gaussian_draw(ph.pst,ph.parcov,num_reals=100)